<a href="https://colab.research.google.com/github/MiladShahabadi/-Text-Classification-Sentiment-Analysis-of-IMDB-Reviews/blob/main/task_for_apply.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# یکم - مدیریت جی پی یو

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## step1: connecting the CoLab and the drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

## step2: installing and setting the darknet

In [ ]:
# Clone Darknet
!git clone https://github.com/AlexeyAB/darknet.git

fatal: destination path 'darknet' already exists and is not an empty directory.


In [ ]:
# Configure Darknet
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

[Errno 20] Not a directory: 'darknet'
/content/darknet


In [ ]:
# Compile DarkNet
!make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv.cpp:1150:10: note: ...thi

## step3:YOLO settings

In [ ]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=24@' cfg/yolov3_training.cfg

## step4: creating files and saving the results

In [ ]:
# Create .names files
!echo -e 'automobile\nambulance\nperson' > data/obj.names


In [ ]:
# Create .data file 
!echo -e 'classes= 3\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

## step5:refer photos to Yolo

In [ ]:
# Create a folder and unzip image dataset
!mkdir data/obj
!unzip /mydrive/yolov3/images.zip -d data/obj

mkdir: cannot create directory ‘data/obj’: File exists
Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/1479498373462797835.jpg  
  inflating: data/obj/1479498373462797835.txt  
  inflating: data/obj/1479498373962951201.jpg  
  inflating: data/obj/1479498373962951201.txt  
  inflating: data/obj/1479498374962942172.jpg  
  inflating: data/obj/1479498374962942172.txt  
  inflating: data/obj/1479498375942206592.jpg  
  inflating: data/obj/1479498375942206592.txt  
  inflating: data/obj/1479498376463086347.jpg  
  inflating: data/obj/1479498376463086347.txt  
  inflating: data/obj/1479498377963597629.jpg  
  inflating: data/obj/1479498377963597629.txt  
  inflating: data/obj/1479498378965237962.jpg  
  inflating: data/obj/1479498378965237962.txt  
  inflating: data/obj/1479498380466064740.jpg  
  inflating: data/obj/1479498380466064740.txt  
  inflating: data/obj/1479498381465380454.jpg  
  inflating: data/obj/1479498381465380454.txt  
  inflating: data/obj/1479498382449249006.jp

In [ ]:
# Create train.txt file
import glob
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

## step6: receiving the pretrained weights and starting Training

In [ ]:
# Download pre-trained weights for the convolutional layers file
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-10-24 09:30:34--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.2’

darknet53.conv.74.2 100%[===================>] 154.96M  44.8MB/s    in 3.8s    

2022-10-24 09:30:38 (40.6 MB/s) - ‘darknet53.conv.74.2’ saved [162482580/162482580]



In [ ]:
# Start training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show  
 

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.786787), count: 29, class_loss = 0.495279, iou_loss = 0.978845, total_loss = 1.474123 
 total_bbox = 1158618, rewritten_bbox = 1.451730 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.821553), count: 2, class_loss = 0.132138, iou_loss = 0.038883, total_loss = 0.171021 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.851335), count: 15, class_loss = 0.127337, iou_loss = 0.291015, total_loss = 0.418351 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.786605), count: 28, class_loss = 0.501812, iou_loss = 0.778420, total_loss = 1.280232 
 total_bbox = 1158663, rewritten_bbox = 1.451759 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.917706), count: 4, class_loss = 0.003434, iou_loss = 0.017205, total_loss = 0.020639 
